# 03 — Resolucion de Conflictos entre Agentes

**Objetivo**: Implementar 3 estrategias para resolver discrepancias entre agentes: votacion, debate, y juez.

## Contenido
1. Votacion por mayoria (3 agentes)
2. Debate con rondas hasta convergencia
3. Juez evaluador
4. Comparacion de estrategias

In [ ]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0.7)  # Temperatura alta para diversidad
llm_judge = ChatOpenAI(model="gpt-5-mini", temperature=0)  # Temperatura baja para juez

print("=" * 60)
print("RESOLUCION DE CONFLICTOS")
print("=" * 60)

## 1. Votacion por Mayoria

3 agentes con perspectivas diferentes votan. La mayoria gana.

In [ ]:
# ============================================================
# ESTRATEGIA 1: VOTACION POR MAYORIA
# ============================================================

class Voto(BaseModel):
    """Voto de un agente."""
    eleccion: str = Field(description="Batman o Spider-Man")
    justificacion: str = Field(description="Justificacion breve del voto")
    confianza: float = Field(description="Confianza en el voto (0-1)", ge=0.0, le=1.0)


perspectivas = {
    "estratega": "Evalua desde la perspectiva de estrategia militar y planificacion tactica.",
    "cientifico": "Evalua desde la perspectiva cientifica, innovacion tecnologica y adaptabilidad.",
    "lider": "Evalua desde la perspectiva de liderazgo, carisma y capacidad de inspirar.",
}

voter_llm = llm.with_structured_output(Voto)


def votacion(pregunta: str) -> dict:
    """Resuelve por votacion de 3 agentes."""
    votos = []
    for perspectiva, descripcion in perspectivas.items():
        voto = voter_llm.invoke(
            f"Perspectiva: {descripcion}\n\n{pregunta}\n\nElige entre Batman o Spider-Man."
        )
        votos.append({"perspectiva": perspectiva, **voto.model_dump()})
    
    # Conteo
    conteo = {}
    for v in votos:
        conteo[v["eleccion"]] = conteo.get(v["eleccion"], 0) + 1
    
    ganador = max(conteo, key=conteo.get)
    
    return {
        "estrategia": "votacion",
        "votos": votos,
        "conteo": conteo,
        "ganador": ganador,
        "unanime": len(set(v["eleccion"] for v in votos)) == 1,
    }


# Test
pregunta_test = "Quien seria mejor lider en una crisis global: Batman o Spider-Man?"
result_voto = votacion(pregunta_test)

print("VOTACION POR MAYORIA")
print(f"Pregunta: {pregunta_test}")
for v in result_voto["votos"]:
    print(f"  [{v['perspectiva']:12s}] {v['eleccion']:12s} (confianza: {v['confianza']:.0%})")
    print(f"                  {v['justificacion'][:100]}")
print(f"Resultado: {result_voto['ganador']} ({result_voto['conteo']})")
print(f"Unanime: {result_voto['unanime']}")

## 2. Debate con Rondas

Dos agentes debaten, presentando argumentos y contra-argumentos, hasta que convergen o se agotan las rondas.

In [ ]:
# ============================================================
# ESTRATEGIA 2: DEBATE
# ============================================================

class ArgumentoDebate(BaseModel):
    """Argumento en un debate."""
    posicion: str = Field(description="Batman o Spider-Man")
    argumento: str = Field(description="Argumento principal")
    contra_argumento: str = Field(description="Respuesta al argumento del oponente")
    cambio_posicion: bool = Field(description="Si cambio de posicion tras los argumentos del oponente")

debate_llm = llm.with_structured_output(ArgumentoDebate)


def debate(pregunta: str, max_rondas: int = 3) -> dict:
    """Resuelve por debate entre dos agentes."""
    historial = []
    posiciones = {"defensor_batman": "Batman", "defensor_spiderman": "Spider-Man"}
    
    for ronda in range(max_rondas):
        ronda_args = {}
        for agente, posicion_inicial in posiciones.items():
            historial_texto = "\n".join([
                f"Ronda {h['ronda']}, {h['agente']}: {h['argumento'][:150]}"
                for h in historial
            ]) if historial else "Sin historial previo."
            
            arg = debate_llm.invoke(
                f"Eres {agente} defendiendo a {posicion_inicial}.\n"
                f"Pregunta: {pregunta}\n\n"
                f"Historial del debate:\n{historial_texto}\n\n"
                f"Presenta tu argumento y responde al oponente."
            )
            
            ronda_args[agente] = arg
            historial.append({
                "ronda": ronda + 1,
                "agente": agente,
                "posicion": arg.posicion,
                "argumento": arg.argumento,
                "cambio": arg.cambio_posicion,
            })
        
        # Verificar convergencia
        posiciones_actuales = [a.posicion for a in ronda_args.values()]
        if len(set(posiciones_actuales)) == 1:
            return {
                "estrategia": "debate",
                "rondas": ronda + 1,
                "convergencia": True,
                "ganador": posiciones_actuales[0],
                "historial": historial,
            }
    
    # Sin convergencia: la posicion del ultimo argumento
    return {
        "estrategia": "debate",
        "rondas": max_rondas,
        "convergencia": False,
        "ganador": "Sin consenso",
        "historial": historial,
    }


result_debate = debate(pregunta_test)

print("\nDEBATE")
for h in result_debate["historial"]:
    cambio = " [CAMBIO]" if h["cambio"] else ""
    print(f"  Ronda {h['ronda']}, {h['agente']:25s}: {h['posicion']}{cambio}")
    print(f"    {h['argumento'][:120]}...")
print(f"Resultado: {result_debate['ganador']} (convergencia: {result_debate['convergencia']}, rondas: {result_debate['rondas']})")

## 3. Juez Evaluador

Un tercer agente evalua los argumentos de ambos lados y emite un veredicto.

In [ ]:
# ============================================================
# ESTRATEGIA 3: JUEZ
# ============================================================

class Veredicto(BaseModel):
    """Veredicto del juez."""
    ganador: str = Field(description="Batman o Spider-Man")
    score_batman: int = Field(description="Score de Batman (1-10)", ge=1, le=10)
    score_spiderman: int = Field(description="Score de Spider-Man (1-10)", ge=1, le=10)
    razonamiento: str = Field(description="Razonamiento detallado del juez")
    aspectos_evaluados: list[str] = Field(description="Aspectos que se evaluaron")

judge_structured = llm_judge.with_structured_output(Veredicto)


def juez(pregunta: str) -> dict:
    """Resuelve usando un juez que evalua argumentos de ambos lados."""
    # Generar argumentos de cada lado
    arg_batman = llm.invoke([
        SystemMessage(content="Presenta los mejores argumentos a favor de Batman."),
        HumanMessage(content=pregunta),
    ])
    
    arg_spider = llm.invoke([
        SystemMessage(content="Presenta los mejores argumentos a favor de Spider-Man."),
        HumanMessage(content=pregunta),
    ])
    
    # Juez evalua
    veredicto = judge_structured.invoke(
        f"Pregunta: {pregunta}\n\n"
        f"Argumentos a favor de Batman:\n{arg_batman.content}\n\n"
        f"Argumentos a favor de Spider-Man:\n{arg_spider.content}\n\n"
        f"Evalua ambos argumentos de forma imparcial y emite un veredicto."
    )
    
    return {
        "estrategia": "juez",
        "veredicto": veredicto.model_dump(),
        "ganador": veredicto.ganador,
        "arg_batman": arg_batman.content[:300],
        "arg_spider": arg_spider.content[:300],
    }


result_juez = juez(pregunta_test)

print("\nJUEZ EVALUADOR")
v = result_juez["veredicto"]
print(f"  Batman: {v['score_batman']}/10")
print(f"  Spider-Man: {v['score_spiderman']}/10")
print(f"  Ganador: {v['ganador']}")
print(f"  Aspectos: {v['aspectos_evaluados']}")
print(f"  Razonamiento: {v['razonamiento'][:300]}...")

## 4. Comparacion de Estrategias

In [ ]:
# ============================================================
# COMPARACION CON 5 PREGUNTAS
# ============================================================

preguntas_controversiales = [
    "Quien tiene mejor filosofia moral: Batman o Spider-Man?",
    "Quien es mejor detective: Batman o Spider-Man?",
    "Quien maneja mejor la perdida personal?",
    "Quien tiene mejores aliados y por que?",
    "Quien seria mas efectivo en el universo del otro?",
]

print("=" * 60)
print("COMPARACION DE ESTRATEGIAS (5 preguntas)")
print("=" * 60)

resultados = []
for pregunta in preguntas_controversiales:
    print(f"\nQ: {pregunta}")
    
    t0 = time.time()
    r_voto = votacion(pregunta)
    lat_voto = (time.time() - t0) * 1000
    
    t0 = time.time()
    r_juez = juez(pregunta)
    lat_juez = (time.time() - t0) * 1000
    
    print(f"  Votacion: {r_voto['ganador']:12s} (unanime: {r_voto['unanime']}, {lat_voto:.0f}ms)")
    print(f"  Juez:     {r_juez['ganador']:12s} ({lat_juez:.0f}ms)")
    
    resultados.append({
        "pregunta": pregunta[:40],
        "votacion": r_voto["ganador"],
        "juez": r_juez["ganador"],
        "acuerdo": r_voto["ganador"] == r_juez["ganador"],
    })

acuerdos = sum(1 for r in resultados if r["acuerdo"])
print(f"\nAcuerdo entre estrategias: {acuerdos}/{len(resultados)}")

## Takeaways

1. **Votacion**: Rapida, democratica, pero puede fallar si los votantes tienen sesgos similares
2. **Debate**: Explora argumentos en profundidad, pero costoso en tokens y tiempo
3. **Juez**: Balance entre calidad y costo, pero depende de un solo punto de vista
4. Ninguna estrategia es universalmente superior — elegir segun el caso de uso
5. El **acuerdo entre estrategias** es una senal de robustez en la decision
6. Para produccion, combinar votacion rapida + juez en caso de empate